In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import get_data
import statsmodels
from simulate import simulatedata

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
syms = ['BTCUSDT']
freq = '10min'
start_date = '2020-01-01'
end_date = '2020-01-03'

In [4]:
raw_data = get_data.get_data(syms, freq, start_date, end_date)
raw_data

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=695704595703-1kpq4gcdb9769asd1vhdo80fssdia4m6.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


c:\Users\admin\Desktop\Binance\Binance\get_data.py:52: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  data[symbol] = _load_df_from_file(symbol, frequency)[start_time:end_time]


{'BTCUSDT':                               open     high      low    close      volume  \
 time                                                                        
 2020-01-01 00:00:00+00:00  7195.24  7196.25  7178.20  7191.07  154.874358   
 2020-01-01 00:10:00+00:00  7193.15  7193.53  7177.35  7178.29   80.261439   
 2020-01-01 00:20:00+00:00  7177.71  7185.56  7175.47  7178.45   96.049725   
 2020-01-01 00:30:00+00:00  7178.19  7185.44  7176.23  7177.53   59.973326   
 2020-01-01 00:40:00+00:00  7177.14  7182.99  7176.34  7182.94   44.872982   
 ...                            ...      ...      ...      ...         ...   
 2020-01-02 23:20:00+00:00  6959.16  6962.39  6950.00  6962.34  217.262815   
 2020-01-02 23:30:00+00:00  6962.47  6974.75  6957.66  6965.37  145.757951   
 2020-01-02 23:40:00+00:00  6965.09  6969.46  6959.87  6967.41   75.157310   
 2020-01-02 23:50:00+00:00  6967.40  6968.82  6961.24  6965.71   82.459841   
 2020-01-03 00:00:00+00:00  6965.49  6965.71  6948.45

In [5]:
dt = pd.DataFrame()
dt['close'] = raw_data['BTCUSDT']['close']
dt['ret'] = raw_data['BTCUSDT']['close'].pct_change()
dt

,close,ret
time,,
2020-01-01 00:00:00+00:00,7191.07,NaN
2020-01-01 00:10:00+00:00,7178.29,-0.001777
2020-01-01 00:20:00+00:00,7178.45,0.000022
2020-01-01 00:30:00+00:00,7177.53,-0.000128
2020-01-01 00:40:00+00:00,7182.94,0.000754
...,...,...
2020-01-02 23:20:00+00:00,6962.34,0.000457
2020-01-02 23:30:00+00:00,6965.37,0.000435
2020-01-02 23:40:00+00:00,6967.41,0.000293


In [6]:
order, seasonal_order = simulatedata.get_order(dt['ret'][1:])
order, seasonal_order

((2, 0, 0), (0, 0, 0, 1))

In [ ]:
if sum(seasonal_order) == 1: seasonal_order = (0, 0, 0, 0)
model_params = simulatedata.fit_sarima(dt['ret'][1:], order, seasonal_order)

In [ ]:
t = simulatedata.simulate_sarima(dt['ret'][1:], order, seasonal_order, model_params, len(dt), 10)
t

In [ ]:
simulated_price_data = simulatedata.construct_price_series(t, dt['close'][0], dt.index[0], freq)

In [ ]:
simulatedata.Evaluate_performance(dt['close'], simulated_price_data, lags=30)